# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

from azureml.widgets import RunDetails
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'tguner-automl'

experiment=Experiment(ws, experiment_name)

### Readong the dataset

This dataset is a Mobile Price Classification dataset taken from the Kaggle. It contains various features of the cell phones and tries to predict corresponding mobile phone price range. 0(low cost), 1(medium cost), 2(high cost) and 3(very high cost)
[Kaggle Dataset Link]('https://www.kaggle.com/iabhishekofficial/mobile-price-classification')

In [3]:
key = "Mobile Phone Data"

dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe()
df.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "tugrul-comp"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="price_range",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on tugrul-comp with default configuration
Running on remote compute: tugrul-comp


Experiment,Id,Type,Status,Details Page,Docs Page
tguner-automl,AutoML_61dc40bb-1e30-4976-ad46-ce0b0fe8d0dd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|3                             |
+------------------------------+

********************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run, best_model = remote_run.get_output()
print(best_run)
print(best_model)

Package:azureml-automl-runtime, training version:1.38.0, current version:1.37.0
Package:azureml-core, training version:1.38.0, current version:1.37.0
Package:azureml-dataprep, training version:2.26.0, current version:2.25.0
Package:azureml-dataprep-rslex, training version:2.2.0, current version:2.1.0
Package:azureml-dataset-runtime, training version:1.38.0, current version:1.37.0
Package:azureml-defaults, training version:1.38.0, current version:1.37.0
Package:azureml-interpret, training version:1.38.0, current version:1.37.0
Package:azureml-mlflow, training version:1.38.0, current version:1.37.0
Package:azureml-pipeline-core, training version:1.38.0, current version:1.37.0
Package:azureml-responsibleai, training version:1.38.0, current version:1.37.0
Package:azureml-telemetry, training version:1.38.0, current version:1.37.0
Package:azureml-train-automl-client, training version:1.38.0, current version:1.37.0
Package:azureml-train-automl-runtime, training version:1.38.0, current version

Run(Experiment: tguner-automl,
Id: AutoML_61dc40bb-1e30-4976-ad46-ce0b0fe8d0dd_29,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                ('XGBoostClassifier',
                 XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.5, gamma=0, max_depth=6, max_leaves=63, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=0.4166666666666667, subsample=0.6, tree_method='auto'))],
         verbose=False)


In [9]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/01fbc0ca/classes.interpret.json',
 'explanation/01fbc0ca/expected_values.interpret.json',
 'explanation/01fbc0ca/features.interpret.json',
 'explanation/01fbc0ca/global_names/0.interpret.json',
 'explanation/01fbc0ca/global_rank/0.interpret.json',
 'explanation/01fbc0ca/global_values/0.interpret.json',
 'explanation/01fbc0ca/local_importance_values.interpret.json',
 'explanation/01fbc0ca/per_class_names/0.interpret.json',
 'explanation/01fbc0ca/per_class_rank/0.interpret.json',
 'explanation/01fbc0ca/per_class_values/0.interpret.json',
 'explanation/01fbc0ca/rich_metadata.interpret.json',
 'explanation/01fbc0ca/true_ys_viz.interpret.json',
 'explanation/01fbc0ca/visualization_dict.interpret.json',
 'explanation/01fbc0ca/ys_pred_proba_viz.interpret.json',
 'explanation/01fbc0ca/ys_pred_viz.interpret.json',
 'explanation/eab44b86/classes.interpret.json',
 'explanation/eab44b86/eval_data_viz.interpret.json',
 'expl

In [65]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', './conda_env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.


TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
import joblib
model_name = best_run.properties['model_name']
joblib.dump(best_model, 'automl_model.pkl')
model = remote_run.register_model(model_name=model_name, description='AutoML model', tags=None)

In [14]:

from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core import Environment


inference_config = InferenceConfig(entry_script='./score.py')

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=2, 
                                               )

service = Model.deploy(workspace=ws, 
                           name='tguner-deploy', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig,
                           overwrite=True)

service.wait_for_deployment(True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-12 21:20:40+00:00 Creating Container Registry if not exists.
2022-02-12 21:20:41+00:00 Use the existing image.
2022-02-12 21:20:41+00:00 Generating deployment configuration.
2022-02-12 21:20:42+00:00 Submitting deployment to compute.
2022-02-12 21:20:45+00:00 Checking the status of deployment tguner-deploy..
2022-02-12 21:23:10+00:00 Checking the status of inference endpoint tguner-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
import requests
import json

data = df[:5].drop(['price_range'], axis=1).to_json(orient='records')
service.run(data)


'{"Prediction": [1, 2, 2, 2, 1]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [16]:
print(service.get_logs())
service.delete()

2022-02-12T21:22:56,783273700+00:00 - gunicorn/run 
2022-02-12T21:22:56,783362700+00:00 - rsyslog/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-02-12T21:22:56,790036800+00:00 - iot-server/run 
2022-02-12T21:22:56,812483400+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_6797cf9b513e59b405ce80f3e9222a7d/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-02-12T21:22:57,018451800+00:00 - iot-server/finish 1 0
2022-02-12T21:22:57,019912300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (78)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 107
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-02-12 21:22:59,661 | root | INFO | Starting up app insights client
logging socket was

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
